# TF-MoDISco Analysis for Splice Site Attribution Motifs

This notebook demonstrates the flexible API for discovering DNA motifs using TF-MoDISco-lite on model attributions.

We use tfmodisco-lite to:
1. Analyze which sequence patterns the model uses for splice site prediction
2. Discover interpretable motifs across different conditions and site types
3. Compare motif usage patterns across tissues and donor/acceptor sites
4. Visualize position weight matrices and enrichment statistics

## Load data and set up environment

Import TF-MoDISco-lite, attribution utilities, and visualization libraries. Configure device and random seeds for reproducibility.

In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import json

# Setup path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from splicevo.model import SplicevoModel
from splicevo.utils.data_utils import load_processed_data, load_predictions
from splicevo.utils.model_utils import load_model_and_config
from splicevo.attributions.modisco_analysis import (
    ModiscoConfig, ModiscoInput, ModiscoAnalyzer, AttributionAggregator, 
    analyze_saved_attributions_quick,
)

# Setup device and reproducibility
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)
np.random.seed(42)

print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")

/home/elek/miniforge3/envs/splicevo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
PyTorch version: 2.8.0
NumPy version: 2.2.6


Load model and test data.

In [2]:
# Configuration
species = "mouse_rat_human"
checkpoint_path = f"/home/elek/sds/sd17d003/Anamaria/splicevo/models/full_{species}_weighted_mse/best_model.pt"  
data_path = f"/home/elek/sds/sd17d003/Anamaria/splicevo/data/splits_full/{species}/test/"
predictions_path = f"/home/elek/sds/sd17d003/Anamaria/splicevo/predictions/full_{species}_weighted_mse/"

# Load model and data
print("Loading model and data...")
model, model_config = load_model_and_config(checkpoint_path, device=device)
sequences, labels, _, _, usage, _ = load_processed_data(data_path, verbose=False)
metadata = pd.read_csv(data_path + "metadata.csv")

# Load predictions
# pred_preds, pred_probs, pred_sse, meta, _, _ = load_predictions(predictions_path, verbose=False)

print(f"  Sequences shape: {sequences.shape}")
print(f"  Labels shape: {labels.shape}")
print(f"  Usage shape: {usage.shape}")

Loading model and data...
  Sequences shape: (58823, 1900, 4)
  Labels shape: (58823, 1000)
  Usage shape: (58823, 1000, 99)


Load computed attributions.

In [3]:
window = 400
n_sequences = 100
window_dir = f"{species}_weighted_mse_window_{str(window)}_subset_0-{str(n_sequences)}/"
attribution_dir = f"/home/elek/sds/sd17d003/Anamaria/splicevo/attributions/{window_dir}/"

# Load splice attributions
print("Loading splice attributions...")
splice_attributions = np.load(attribution_dir + "splice_attributions.npy")
splice_attributions_seq = np.load(attribution_dir + "splice_sequences.npy")
splice_attributions_meta_path = attribution_dir + "splice_metadata.json"
with open(splice_attributions_meta_path, 'r') as f:
    splice_attributions_meta = json.load(f)

print(f"  Splice attributions shape: {splice_attributions.shape}")
print(f"  Splice sequences shape: {splice_attributions_seq.shape}")

# Load usage attributions
print("Loading usage attributions...")
usage_attributions = np.load(attribution_dir + "usage_attributions.npy")
usage_attributions_seq = np.load(attribution_dir + "usage_sequences.npy")
usage_attributions_meta_path = attribution_dir + "usage_metadata.json"
with open(usage_attributions_meta_path, 'r') as f:
    usage_attributions_meta = json.load(f)

print(f"  Usage attributions shape: {usage_attributions.shape}")
print(f"  Usage sequences shape: {usage_attributions_seq.shape}")

# Get condition names
with open(predictions_path + "metadata.json", 'r') as f:
    meta_json = json.load(f)
    condition_names = meta_json.get('conditions', None)

print(f"  Number of conditions: {len(condition_names)}")
print(f"  Conditions: {condition_names[:5]}...")

Loading splice attributions...
  Splice attributions shape: (180, 401, 4)
  Splice sequences shape: (180, 401, 4)
Loading usage attributions...
  Usage attributions shape: (17820, 401, 4)
  Usage sequences shape: (17820, 401, 4)
  Number of conditions: 99
  Conditions: ['Brain_1', 'Brain_10', 'Brain_11', 'Brain_12', 'Brain_13']...


## Run TF-MoDISco

In [4]:
import os
modisco_dir = Path.home() / f"sds/sd17d003/Anamaria/splicevo/tfmodisco/"
os.makedirs(modisco_dir, exist_ok = True)

Create modisco configuration.

In [5]:
# Configure TF-MoDISco-lite parameters with different settings than the quick start
config = ModiscoConfig(
    sliding_window_size=12,
    flank_size=5,
    min_passing_windows_frac=0.03,
    max_passing_windows_frac=0.2,
    target_seqlet_fdr=0.01,
    trim_to_window_size=12,
    initial_flank_to_add=5,
    n_processing_cores=4,
)

Initialize ModiscoAnalyzer with config.

In [6]:
# Initialize analyzer with the custom config
manual_analyzer = ModiscoAnalyzer(config=config, verbose=True)

### TF-MoDISco for splice site attributions

Run motif discovery on acceptor and donor attributions separatelly.

In [ ]:
# Run analysis on saved splice attributions using the new API
from splicevo.attributions import analyze_saved_attributions_quick

# Use the saved splice attributions
splice_base_path = attribution_dir + "splice"

# Quick analysis by site type
print("Running TF-MoDISco analysis on splice attributions by site type...")
modisco_out_dir = modisco_dir / f"subset{species}_weighted_mse_window_{str(config.trim_to_window_size)}_flank_{str(config.initial_flank_to_add)}_fdr_{str(config.target_seqlet_fdr)}_subset_0-{str(n_sequences)}"
analyzer = analyze_saved_attributions_quick(
    base_path=splice_base_path,
    aggregation='by_site_type',
    output_dir=modisco_out_dir / "splice_by_site_type",
    config=config,
    verbose=True
)

Running TF-MoDISco analysis on splice attributions by site type...
Loading saved attributions from /home/elek/sds/sd17d003/Anamaria/splicevo/attributions/mouse_rat_human_weighted_mse_window_400_subset_0-100/splice
  Sequences shape: (180, 401, 4)
  Attributions shape: (180, 401, 4)
  Loaded 180 sites
Aggregating 180 attributions by site type with metadata
  acceptor: 86 samples
  donor: 94 samples

Processing group: 'acceptor'

Running tfmodisco on 'acceptor'
  Samples: 86
  Config: sliding_window_size=12,flank_size=5,min_passing_windows_frac=0.03,max_passing_windows_frac=0.2,min_metacluster_size=20,max_seqlets_per_metacluster=20000 target_seqlet_fdr=0.01 trim_to_window_size=12 initial_flank_to_add=5 final_flank_to_add=0 n_processing_cores=4 batch_size=32
  Preparing modisco input:
  Input shapes: attr=(86, 401, 4), seq=(86, 401, 4)
  Output shapes: attr=(86, 401, 4), seq=(86, 401, 4)
  Attribution range: [-6.8222, 11.9284]
  Input shapes: attr=(86, 401, 4), seq=(86, 401, 4)
  Running 

Examine the motifs discovered by TF-MoDISco and view summary statistics.

In [8]:
# Get summary of analysis results
summary = analyzer.get_summary()
for name, info in summary.items():
    print(f"\n{name}:")
    for key, value in info.items():
        print(f"  {key}: {value}")



acceptor:
  status: complete
  n_samples: 86
  aggregation: by_site_type
  has_motifs: False

donor:
  status: complete
  n_samples: 94
  aggregation: by_site_type
  has_motifs: False


In [9]:
# Display motif information from results
for name, result in analyzer.results.items():
    print(f"\n{name}:")
    print(f"  Status: {result['status']}")
    
    if result['status'] == 'complete':
        motif_summary = result.get('motif_summary', {})
        print(f"  Positive patterns: {motif_summary.get('n_pos_patterns', 0)}")
        print(f"  Negative patterns: {motif_summary.get('n_neg_patterns', 0)}")
        
        if result.get('h5_path'):
            print(f"  HDF5 file: {result['h5_path']}")
    else:
        print(f"  Error: {result.get('error', 'Unknown error')}")



acceptor:
  Status: complete
  Positive patterns: 3
  Negative patterns: 3
  HDF5 file: /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/acceptor_motifs.h5

donor:
  Status: complete
  Positive patterns: 4
  Negative patterns: 4
  HDF5 file: /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/donor_motifs.h5


Extract detailed motif information from HDF5 files and generate visualization reports.

In [10]:
# Extract motifs from HDF5 files
motif_results = {}
for name, result in analyzer.results.items():
    if result.get('h5_path') and result['status'] == 'complete':
        try:
            h5_path = result['h5_path']
            print(f"\nExtracting from {name}...")
            
            motifs = analyzer.extract_motifs_from_h5(h5_path)
            motif_results[name] = motifs
            
            print(f"    Extracted {len(motifs['pos_patterns'])} positive motifs")
            print(f"    Extracted {len(motifs['neg_patterns'])} negative motifs")
            
            # Show details about first few motifs
            if motifs['pos_patterns']:
                print(f"\n  First positive motif:")
                first_motif = motifs['pos_patterns'][0]
                print(f"    Name: {first_motif['name']}")
                print(f"    N seqlets: {first_motif.get('n_seqlets', 'N/A')}")
                if first_motif['sequence'] is not None:
                    print(f"    Sequence shape: {first_motif['sequence'].shape}")
        except Exception as e:
            print(f"  ⚠ Error extracting motifs: {str(e)}")
    elif not result.get('h5_path'):
        print(f"\n{name}: No HDF5 file saved")
    else:
        print(f"\n{name}: Analysis failed (status: {result['status']})")



Extracting from acceptor...
Extracted 3 positive and 3 negative motifs from /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/acceptor_motifs.h5
    Extracted 3 positive motifs
    Extracted 3 negative motifs

  First positive motif:
    Name: pattern_0
    N seqlets: 25
    Sequence shape: (22, 4)

Extracting from donor...
Extracted 4 positive and 4 negative motifs from /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/donor_motifs.h5
    Extracted 4 positive motifs
    Extracted 4 negative motifs

  First positive motif:
    Name: pattern_0
    N seqlets: 27
    Sequence shape: (22, 4)


In [11]:
# Generate HTML reports from modisco results
for name, result in analyzer.results.items():
    if result.get('h5_path') and result['status'] == 'complete':
        h5_path = result['h5_path']
        report_dir = modisco_out_dir / f"{name}_report"
        analyzer.generate_reports(h5_path, report_dir)

Generating reports in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/acceptor_report...


✓ Reports generated in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/acceptor_report
Generating reports in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/donor_report...
✓ Reports generated in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01_subset_0-100/donor_report


Save all analysis results to disk for later reference and comparison.

### TF-MoDISco for usage attributions

In [ ]:
# Run analysis on saved splice attributions using the new API
from splicevo.attributions import analyze_saved_attributions_quick

# Use the saved splice attributions
splice_base_path = attribution_dir + "usage"

# Quick analysis by site type
print("Running TF-MoDISco analysis on splice usage attributions by condition...")
modisco_out_dir = modisco_dir / f"subset_{species}_weighted_mse_window_{str(config.sliding_window_size)}_flank_{str(config.flank_size)}_fdr_{str(config.target_seqlet_fdr)}"
analyzer = analyze_saved_attributions_quick(
    base_path=splice_base_path,
    aggregation='by_condition',
    output_dir=modisco_out_dir / "usage_by_condition",
    config=config,
    verbose=True
)

Running TF-MoDISco analysis on splice usage attributions by condition...
Loading saved attributions from /home/elek/sds/sd17d003/Anamaria/splicevo/attributions/mouse_rat_human_weighted_mse_window_400_subset_0-100/usage
  Sequences shape: (17820, 401, 4)
  Attributions shape: (17820, 401, 4)
  Loaded 17820 sites
Aggregating 17820 attributions by condition
  Brain_1: 180 samples
  Brain_10: 180 samples
  Brain_11: 180 samples
  Brain_12: 180 samples
  Brain_13: 180 samples
  Brain_14: 180 samples
  Brain_2: 180 samples
  Brain_3: 180 samples
  Brain_4: 180 samples
  Brain_5: 180 samples
  Brain_6: 180 samples
  Brain_7: 180 samples
  Brain_8: 180 samples
  Brain_9: 180 samples
  Cerebellum_10: 180 samples
  Cerebellum_11: 180 samples
  Cerebellum_12: 180 samples
  Cerebellum_13: 180 samples
  Cerebellum_14: 180 samples
  Cerebellum_15: 180 samples
  Cerebellum_2: 180 samples
  Cerebellum_3: 180 samples
  Cerebellum_4: 180 samples
  Cerebellum_5: 180 samples
  Cerebellum_6: 180 samples
  

In [13]:
# Get summary of analysis results
summary = analyzer.get_summary()
for name, info in summary.items():
    print(f"\n{name}:")
    for key, value in info.items():
        print(f"  {key}: {value}")


Brain_11:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Brain_12:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Brain_13:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Brain_2:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Brain_3:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Brain_6:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Brain_8:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Cerebellum_10:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Cerebellum_11:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Cerebellum_12:
  status: complete
  n_samples: 180
  aggregation: by_condition
  has_motifs: False

Cerebellum_13:
  status: complete
  n_

In [14]:
# Display motif information from results
for name, result in analyzer.results.items():
    print(f"\n{name}:")
    print(f"  Status: {result['status']}")
    
    if result['status'] == 'complete':
        motif_summary = result.get('motif_summary', {})
        print(f"  Positive patterns: {motif_summary.get('n_pos_patterns', 0)}")
        print(f"  Negative patterns: {motif_summary.get('n_neg_patterns', 0)}")
        
        if result.get('h5_path'):
            print(f"  HDF5 file: {result['h5_path']}")
    else:
        print(f"  Error: {result.get('error', 'Unknown error')}")


Brain_11:
  Status: complete
  Positive patterns: 10
  Negative patterns: 12
  HDF5 file: /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_11_motifs.h5

Brain_12:
  Status: complete
  Positive patterns: 12
  Negative patterns: 15
  HDF5 file: /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_12_motifs.h5

Brain_13:
  Status: complete
  Positive patterns: 11
  Negative patterns: 13
  HDF5 file: /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_13_motifs.h5

Brain_2:
  Status: complete
  Positive patterns: 9
  Negative patterns: 14
  HDF5 file: /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_2_motifs.h5

Brain_3:
  Status: complete
  Positive patterns: 10
  Negative patterns: 14
  HDF5 file: /home/elek/sd

In [15]:
# Extract motifs from HDF5 files
motif_results = {}
for name, result in analyzer.results.items():
    if result.get('h5_path') and result['status'] == 'complete':
        try:
            h5_path = result['h5_path']
            print(f"\nExtracting from {name}...")
            
            motifs = analyzer.extract_motifs_from_h5(h5_path)
            motif_results[name] = motifs
            
            print(f"    Extracted {len(motifs['pos_patterns'])} positive motifs")
            print(f"    Extracted {len(motifs['neg_patterns'])} negative motifs")
            
            # Show details about first few motifs
            if motifs['pos_patterns']:
                print(f"\n  First positive motif:")
                first_motif = motifs['pos_patterns'][0]
                print(f"    Name: {first_motif['name']}")
                print(f"    N seqlets: {first_motif.get('n_seqlets', 'N/A')}")
                if first_motif['sequence'] is not None:
                    print(f"    Sequence shape: {first_motif['sequence'].shape}")
        except Exception as e:
            print(f"    Error extracting motifs: {str(e)}")
    elif not result.get('h5_path'):
        print(f"\n{name}: No HDF5 file saved")
    else:
        print(f"\n{name}: Analysis failed (status: {result['status']})")


Extracting from Brain_11...
Extracted 10 positive and 12 negative motifs from /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_11_motifs.h5
    Extracted 10 positive motifs
    Extracted 12 negative motifs

  First positive motif:
    Name: pattern_0
    N seqlets: 58
    Sequence shape: (22, 4)

Extracting from Brain_12...
Extracted 12 positive and 15 negative motifs from /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_12_motifs.h5
    Extracted 12 positive motifs
    Extracted 15 negative motifs

  First positive motif:
    Name: pattern_0
    N seqlets: 62
    Sequence shape: (22, 4)

Extracting from Brain_13...
Extracted 11 positive and 13 negative motifs from /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_13_motifs.h5
    Extracted 11 positive motifs
    Extracted 

In [16]:
# Generate HTML reports from modisco results
for name, result in analyzer.results.items():
    if result.get('h5_path') and result['status'] == 'complete':
        h5_path = result['h5_path']
        report_dir = modisco_out_dir / f"{name}_report"
        analyzer.generate_reports(h5_path, report_dir)

Generating reports in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_11_report...


✓ Reports generated in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_11_report
Generating reports in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_12_report...
✓ Reports generated in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_12_report
Generating reports in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_13_report...
✓ Reports generated in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_13_report
Generating reports in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodisco/subset_mouse_rat_human_weighted_mse_window_12_flank_5_fdr_0.01/Brain_2_report...
✓ Reports generated in /home/elek/sds/sd17d003/Anamaria/splicevo/tfmodi